# Simple User Item Biases
* Computes a bias for each user and for each item
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = m + u_i + a_j$
* $m = \text{mean}_{ij}(r_{ij})$
* $u_i = \text{mean}_j(r_{ij}) - m$
* $a_j = \text{mean}_i(r_{ij}) - m$
* $r_{ij}$ is the rating for user $i$ and item $j$
* Useful as a preprocessing step for other alphas

In [1]:
name = "SimpleUserItemBiases";
residual_alphas = [];

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [3]:
using DataStructures

## Training

In [4]:
training_df =
    DataFrame(user = training.user, item = training.item, rating = training.rating);

In [5]:
μ = mean(training.rating);

In [6]:
u = combine(groupby(training_df, :user), :rating => mean => :rating)
u = DefaultDict(0.0, Dict(Pair.(u.user, u.rating .- μ)));

In [7]:
a = combine(groupby(training_df, :item), :rating => mean => :rating)
a = DefaultDict(0.0, Dict(Pair.(a.item, a.rating .- μ)));

## Inference

In [8]:
function make_prediction(users, items, u, a, μ)
    r = zeros(eltype(μ), length(users))
    for i = 1:length(r)
        r[i] = u[users[i]] + a[items[i]] + μ
    end
    r
end;

model(users, items) = make_prediction(users, items, u, a, μ);

In [9]:
write_predictions(model);

[ Info: 20211204 20:11:50 training set: RMSE 1.3329115279899955 MAE 0.9997747594835902 R2 0.4395862105349557
[ Info: 20211204 20:11:52 validation set: RMSE 1.342890864774748 MAE 1.0067577667022556 R2 0.4312189466062175


In [10]:
write_params(Dict("u" => u, "a" => a));